In [1]:
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
%cd Qwen3-TTS/finetuning


In [ ]:
import json
import soundfile as sf

ref_dict = {"Lisa": "C:/Users/User/Comvi/Voice/Lisa/dataset_denoised/gEMrqw-pAy4/SPEAKER_01_sent_0001_DeepFilterNet3.wav",
            "Bambam": "C:/Users/User/Comvi/Voice/Bambam/dataset_denoised/zbo7Mk3ryaI/SPEAKER_00_sent_0001_DeepFilterNet3.wav",
            "IU": "C:/Users/User/Comvi/Voice/IU/dataset_denoised/wCbUWU4l_Ko/SPEAKER_01_sent_0000_DeepFilterNet3.wav",
            "IVE": "C:/Users/User/Comvi/Voice/IVE/dataset_denoised/_037bSnAyRg/SPEAKER_01_sent_0007_DeepFilterNet3.wav"
            }

manifest_path = "C:/Users/User/Comvi/Voice/manifest_whisper_text.jsonl"
output_path = "manifest_whisper_text.jsonl"

with open(manifest_path, "r", encoding="utf-8") as f_in, \
     open(output_path, "w", encoding="utf-8") as f_out:

    for line in f_in:
        data_dict = json.loads(line.strip())

        if data_dict["speaker"] != "Lisa":
            continue
        

        wav_path = data_dict["audio"]
        new_path = wav_path.replace("C:/Users/User/Comvi/Voice/", "../../")
        data_dict["audio"] = new_path

        ref_path = ref_dict[f"{data_dict['speaker']}"]
        new_ref_path = ref_path.replace("C:/Users/User/Comvi/Voice/", "../../")
        data_dict["ref_audio"] = new_ref_path

        # เขียนกลับเป็น json 1 บรรทัด
        f_out.write(json.dumps(data_dict, ensure_ascii=False) + "\n")

print("Done. Saved to:", output_path)

manifest_path = "C:/Users/User/Comvi/Voice/manifest_pathumma_text.jsonl"
output_path = "manifest_pathumma_text.jsonl"

with open(manifest_path, "r", encoding="utf-8") as f_in, \
     open(output_path, "w", encoding="utf-8") as f_out:

    for line in f_in:
        data_dict = json.loads(line.strip())

        if data_dict["speaker"] != "Lisa":
            continue

        wav_path = data_dict["audio"]
        new_path = wav_path.replace("C:/Users/User/Comvi/Voice/", "../../")
        data_dict["audio"] = new_path

        ref_path = ref_dict[f"{data_dict['speaker']}"]
        new_ref_path = ref_path.replace("C:/Users/User/Comvi/Voice/", "../../")
        data_dict["ref_audio"] = new_ref_path

        # เขียนกลับเป็น json 1 บรรทัด
        f_out.write(json.dumps(data_dict, ensure_ascii=False) + "\n")

print("Done. Saved to:", output_path)

Done. Saved to: manifest_whisper_text.jsonl
Done. Saved to: manifest_pathumma_text.jsonl


In [ ]:
!python prepare_data.py --device cuda:0 --tokenizer_model_path Qwen/Qwen3-TTS-Tokenizer-12Hz --input_jsonl "../../manifest_whisper_train_raw.jsonl" --output_jsonl "../../manifest_whisper_train_with_code.jsonl"

In [ ]:
!python prepare_data.py --device cuda:0 --tokenizer_model_path Qwen/Qwen3-TTS-Tokenizer-12Hz --input_jsonl "../../manifest_pathumma_train_raw.jsonl" --output_jsonl "../../manifest_pathumma_train_with_code.jsonl"

In [ ]:
!pip install -U huggingface_hub


In [ ]:
from huggingface_hub import snapshot_download

snapshot_download(
    repo_id="Qwen/Qwen3-TTS-12Hz-1.7B-Base",
    local_dir="./Qwen3-TTS-12Hz-1.7B-Base",
    local_dir_use_symlinks=False
)

In [ ]:
!python sft_12hz.py --init_model_path "./Qwen3-TTS-12Hz-1.7B-Base" --output_model_path "../../output_whisper" --train_jsonl "../../manifest_whisper_train_with_code_fixed.jsonl" --batch_size 1 --lr 1e-6 --num_epochs 10 --speaker_name multi_speaker

In [ ]:
!python sft_12hz.py --init_model_path "./Qwen3-TTS-12Hz-1.7B-Base" --output_model_path "../../output_pathumma" --train_jsonl "../../manifest_pathumma_train_with_code_fixed.jsonl" --batch_size 1 --lr 1e-6 --num_epochs 10 --speaker_name multi_speaker